<a href="https://colab.research.google.com/github/lalopey/2019-CS109A/blob/master/lalo/Gresearch_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture 
!pip install pandas_ta

#G-Research Starter

To Do:

- Reproduce results from leaderboard by local backtesting
- Make vol ratio function more general

Double check that fillna(0) is correct in postprocess

# Setup

## Imports

In [3]:
import numpy as np
import pandas as pd


import pandas_ta as ta
import lightgbm as lgb

from tqdm.notebook import tqdm

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.mode.use_inf_as_na = True


## Paths

In [4]:
DATA_PATH = '/content/drive/MyDrive/gresearch/data/train.parquet'
ASSET_PATH = '/content/drive/MyDrive/gresearch/data/asset_details.csv'
FACTOR_RETURNS_PATH = '/content/drive/MyDrive/gresearch/data/factor_returns.csv'
TIMESTAMP_MIN = 1555222560


## Functions

### Data

In [5]:
def read_data(file_name=DATA_PATH):
    
    df = pd.read_parquet(file_name)
    df = df[df.timestamp >= TIMESTAMP_MIN]
    df = df[~df.Target.isnull()].reset_index(drop=True)

    asset_df = pd.read_csv(ASSET_PATH)
    asset_df['Asset_Ticker'] = ['BCH', 'BNB', 'BTC', 'EOS', 'ETC', 'ETH', 'LTC',
                                'XMR', 'TRON', 'XLM', 'ADA', 'IOTA', 'MKR', 'DOGE']

    return df, asset_df


def preprocess_data(df):

    df = df[['timestamp', 'Asset_ID', 'Close', 'VWAP', 'Volume', 'Count', 'Target']]
    df.loc[df.VWAP.isna(), 'VWAP'] = df['Close'][df.VWAP.isna()]
    df.loc[df.VWAP < 0, 'VWAP'] = df['Close'][df.VWAP < 0]

    df['Dollar'] = df['Volume'] * df['Close'] 

    df[df.Volume.isnull()] = 0
    df[df.Count.isnull()] = 0
    df[df.Dollar.isnull()] = 0

    df['log_VWAP'] = np.log(df['VWAP'])
    df['log_Close'] = np.log(df['Close'])

    return df


def postprocess_data(df):

    df = df[~df.Factor_Target.isnull()].reset_index(drop=True)
    df = df.drop(['log_VWAP', 'log_Close'], axis=1)
    df = df.fillna(0)

    return df


def weighted_correlation(a, b, weights):

  w = np.ravel(weights)
  a = np.ravel(a)
  b = np.ravel(b)

  sum_w = np.sum(w)
  mean_a = np.sum(a * w) / sum_w
  mean_b = np.sum(b * w) / sum_w
  var_a = np.sum(w * np.square(a - mean_a)) / sum_w
  var_b = np.sum(w * np.square(b - mean_b)) / sum_w

  cov = np.sum((a * b * w)) / np.sum(w) - mean_a * mean_b
  corr = cov / np.sqrt(var_a * var_b)

  return corr
  


### Features

In [6]:
def return_factors(df, col_name, periods):

    for period in tqdm(periods):

        new_col = 'Return_{}_{}'.format(col_name, period)
        
        ret_df = df.groupby(['timestamp', 'Asset_ID'])['log_' + col_name].first().unstack() 
        ret_df = ret_df.diff(period).stack().reset_index()
        ret_df = ret_df.rename(columns={0:new_col})

        ret_df[new_col] = ret_df[new_col] - ret_df.groupby('timestamp')[new_col].transform(np.mean)

        df = df.merge(ret_df, on=['timestamp', 'Asset_ID'], how='left')

    return df


def vol_ratio_factors(df):

    windows = [100, 1000, 10000]
    pairs = [(100, 1000), (1000, 10000)]

    for window in tqdm(windows):

        vol_df = df.groupby(['timestamp', 'Asset_ID'])['Return_VWAP_1'].first().unstack().rolling(window).std().stack()
        vol_df.name = 'Vol_VWAP_{}'.format(window)
        vol_df = vol_df.reset_index()
        df = df.merge(vol_df, how='left')
    
    for pair in pairs:

        pair_str = 'Vol_ratio_VWAP_{}_{}'.format(pair[0], pair[1])

        df[pair_str] = df['Vol_VWAP_{}'.format(pair[0])] / df['Vol_VWAP_{}'.format(pair[1])]
        # Note minus sign
        df[pair_str] = - (df[pair_str] - df.groupby('timestamp')[pair_str].transform(np.mean) )

    df = df.drop(['Vol_VWAP_{}'.format(window) for window in windows], axis=1)

    return df


def volume_ratio_factors(df):

    windows = [100, 1000, 10000]
    pairs = [(100, 1000), (1000, 10000)]

    for window in tqdm(windows):

        vol_df = df.groupby(['timestamp', 'Asset_ID'])['Volume'].first().unstack().rolling(window).sum().stack()
        vol_df.name = 'Volume_{}'.format(window)
        vol_df = vol_df.reset_index()
        df = df.merge(vol_df, how='left')
    
    for pair in pairs:

        pair_str = 'Volume_ratio_{}_{}'.format(pair[0], pair[1])

        df[pair_str] = df['Volume_{}'.format(pair[0])] / df['Volume_{}'.format(pair[1])]
        # Note minus sign
        df[pair_str] = - ( df[pair_str] - df.groupby('timestamp')[pair_str].transform(np.mean) )

    df = df.drop(['Volume_{}'.format(window) for window in windows], axis=1)

    return df


def count_ratio_factors(df):

    windows = [100, 1000, 10000]
    pairs = [(100, 1000), (1000, 10000)]

    for window in tqdm(windows):

        vol_df = df.groupby(['timestamp', 'Asset_ID'])['Count'].first().unstack().rolling(window).sum().stack()
        vol_df.name = 'Count_{}'.format(window)
        vol_df = vol_df.reset_index()
        df = df.merge(vol_df, how='left')
    
    for pair in pairs:

        pair_str = 'Count_ratio_{}_{}'.format(pair[0], pair[1])

        df[pair_str] = df['Count_{}'.format(pair[0])] / df['Count_{}'.format(pair[1])]
        # Note minus sign
        df[pair_str] = - (df[pair_str] - df.groupby('timestamp')[pair_str].transform(np.mean))

    df = df.drop(['Count_{}'.format(window) for window in windows], axis=1)

    return df


def pair_factors(df, asset_df):

    pairs = [('BTC','ETH'), ('ETC', 'ETH')]
    asset_df = asset_df.set_index('Asset_Ticker') 

    for pair in pairs:

        pair_str = '{}_{}'.format(pair[0], pair[1])
        df[pair_str] = 0
        df.loc[df.Asset_ID==asset_df.loc[pair[0]].Asset_ID, pair_str] = 1
        df.loc[df.Asset_ID==asset_df.loc[pair[1]].Asset_ID, pair_str] = -1

    return df


def size_factors(df, asset_df):

    asset_df = asset_df.set_index('Asset_Ticker') 

    df['ETHBTC_Rest_HighSize'] = -1
    df.loc[df.Asset_ID.isin([asset_df.loc['BTC'].Asset_ID, 
                             asset_df.loc['ETC'].Asset_ID]), 'ETHBTC_Rest_HighSize'] = 1
    df['ETHBTC_Rest_HighSize'] = df['ETHBTC_Rest_HighSize'] - df.groupby('timestamp')['ETHBTC_Rest_HighSize'].transform(np.mean)

    df['ETHBTC_Rest_All'] = 0
    df.loc[df.Asset_ID.isin([asset_df.loc['TRON'].Asset_ID, 
                             asset_df.loc['ADA'].Asset_ID,
                             asset_df.loc['IOTA'].Asset_ID,
                             asset_df.loc['MKR'].Asset_ID]), 'ETHBTC_Rest_All'] = -1
    df.loc[df.Asset_ID.isin([asset_df.loc['BTC'].Asset_ID, 
                             asset_df.loc['ETC'].Asset_ID]), 'ETHBTC_Rest_All'] = 1
    df['ETHBTC_Rest_All'] = df['ETHBTC_Rest_All'] - df.groupby('timestamp')['ETHBTC_Rest_All'].transform(np.mean)

    return df


def factor_target(df, period=-1):
    
    ret_df = df.groupby(['timestamp', 'Asset_ID'])['log_VWAP'].first().unstack() 
    ret_df = ret_df.diff(period).stack().reset_index()
    ret_df = ret_df.rename(columns={0:'Factor_Target'})

    ret_df = ret_df.groupby(['timestamp', 'Asset_ID'])['Factor_Target'].first()
    ret_df = ret_df.groupby(level=1).shift(period).reset_index()

    df = df.merge(ret_df, on=['timestamp', 'Asset_ID'], how='left')

    return df


def factor_returns(df, factor_cols, save=False):

    ret_df = df[factor_cols].copy()
    ret_df['timestamp'] = df['timestamp']
    ret_df['Asset_ID'] = df['Asset_ID']

    for col in factor_cols:
        ret_df[col] = ret_df[col] * df['Factor_Target']

    ret_df = ret_df.groupby('timestamp')[factor_cols].sum().reset_index()
    # 10000 is the largest sliding window for vol factors
    ret_df = ret_df.iloc[10000:].reset_index(drop=True)

    if save:
        ret_df.to_csv(FACTOR_RETURNS_PATH, index=False)

    return ret_df


def calc_factor_prices(ret_df):
    """
    From factor returns to prices, all factors start at 1
    """
    prices_df = np.exp(ret_df.cumsum())

    return prices_df


def calc_factor_features(ret_df, prices_df):

    ###### Factor return EWMAs #####
    print('Calculating factor return EWMAs')
    feature_df_list = []
    feature_df_cols = []

    feature_df_list.append(ret_df.stack())
    feature_df_cols.append('RETURN')  

    feature_df_list += [ret_df.ewm(com=com).mean().stack(dropna=False) for com in [15, 30, 60, 125, 500, 1000, 2000]]
    feature_df_cols += ['RETURN_EWM{}'.format(com) for com in [15, 30, 60, 125, 500, 1000, 2000]]

    feature_df = pd.concat(feature_df_list, axis=1)
    feature_df.columns = feature_df_cols


    print('Calculating factor price RSIs')
    rsi_df_list = []
    rsi_df_columns = ['RSI_{}'.format(com) for com in [15, 30, 60, 125, 500, 1000, 2000]]

    for period in [15, 30, 60, 125, 500, 1000, 2000]:

        rsi_df = pd.DataFrame()
        

        for feat in prices_df.columns:
            rsi_df[feat] = ta.rsi(prices_df[feat], length=period)
        
        rsi_df_list.append(rsi_df.dropna())

    mintime = max([rs.index[0] for rs in rsi_df_list])
    rsi_df_list = [rs[rs.index>= mintime].stack().rename(rsi_df_columns[i]) for i, rs in enumerate(rsi_df_list)]

    rsi_df = pd.concat(rsi_df_list, axis=1).dropna()

    feature_df = feature_df[feature_df.index.get_level_values(0) >= rsi_df.index[0][0]]
    feature_df = pd.concat([feature_df, rsi_df], axis=1)

    feature_df['Target'] = feature_df.groupby(level=1).RETURN.shift(-1).values
    feature_df = feature_df.dropna().reset_index()
    feature_df = feature_df.rename(columns={'level_1':'Factor_Name'})
    feature_df["Factor"] = pd.factorize(feature_df.Factor_Name)[0]

    return feature_df

### Pipeline

In [7]:
def pipeline(save=False):

    print('Reading Data')
    df, asset_df = read_data()
    df = preprocess_data(df)

    print('Calculating Factor Target')
    df = factor_target(df, period=-1)
    print('Calculating Return Factors')
    return_periods = [1, 15,30,60,120,240]
    df = return_factors(df, col_name='VWAP', periods=return_periods)
    print('Calculating Volatility Ratio Factors')
    df = vol_ratio_factors(df)
    print('Calculating Volume Ratio Factors')
    df = volume_ratio_factors(df)
    print('Calculating Count Ratio Factors')
    df = count_ratio_factors(df)
    print('Calculating Pair Factors')
    df = pair_factors(df, asset_df)
    print('Calculating Size Factors')
    df = size_factors(df, asset_df)

    df = postprocess_data(df)

    print('Calculating Factor returns')
    factor_cols = df.columns[10:]
    ret_df = factor_returns(df, factor_cols, save=save)

    return df, ret_df


def train_test_split(df):

    #1623542400 == '2021-06-13 00:00:00'
    train_df = df[df['timestamp'] < 1623542400]
    test_df = df[df['timestamp'] >= 1623542400]

    return train_df, test_df



# Pipeline

In [8]:
%%time 
asset_df = pd.read_csv(ASSET_PATH)
df, ret_df = pipeline(save=True)
ret_df = pd.read_csv(FACTOR_RETURNS_PATH).set_index('timestamp')
prices_df = calc_factor_prices(ret_df)
feature_df = calc_factor_features(ret_df, prices_df)
#ret_df = ret_df / ret_df.std()


Reading Data
Calculating Factor Target
Calculating Return Factors


  0%|          | 0/6 [00:00<?, ?it/s]

Calculating Volatility Ratio Factors


  0%|          | 0/3 [00:00<?, ?it/s]

Calculating Volume Ratio Factors


  0%|          | 0/3 [00:00<?, ?it/s]

Calculating Count Ratio Factors


  0%|          | 0/3 [00:00<?, ?it/s]

Calculating Pair Factors
Calculating Size Factors
Calculating Factor returns
Calculating factor return EWMAs
Calculating factor price RSIs
CPU times: user 6min 58s, sys: 35.7 s, total: 7min 33s
Wall time: 7min 36s


In [ ]:
(1632181320 - 1623542400)/600000

In [34]:

start = 1610000000 
end = 1623542400 
endv = 2000000000


eval_df_month = feature_df[['timestamp','Target','Factor_Name']]
eval_df_month['pred'] = np.nan

feats = list(feature_df.columns[2:])
feats.remove('Target')
#feats.remove('RETURN')

params1 = {
        'learning_rate': 0.05,        
        'lambda_l1': 3,
        'lambda_l2': 3,
        'num_leaves': 500,
        'min_sum_hessian_in_leaf': 30,
        'feature_fraction': 0.6,
        'feature_fraction_bynode': 0.8,
        'bagging_fraction': 0.97,
        'bagging_freq': 46,
        'min_data_in_leaf': 500,
        'categorical_column':[-1],
        'seed': 43,
        'feature_fraction_seed': 43,
        'bagging_seed': 43,
        'drop_seed': 43,
        'data_random_seed': 43,
        'objective': 'binary',
        'boosting': 'gbdt',
        'verbosity': -1,
        'n_jobs': -1,
    } 

train_idx = feature_df[(feature_df.timestamp > start) & (feature_df.timestamp <= end)].index
val_idx = feature_df[(feature_df.timestamp > end) & (feature_df.timestamp <= endv)].index

x_train = feature_df[feats].loc[train_idx]
x_val = feature_df[feats].loc[val_idx]

y_train = 1* feature_df['Target'][train_idx] > 0
y_val = 1* feature_df['Target'][val_idx] > 0

train_dataset = lgb.Dataset(x_train, y_train) 
val_dataset = lgb.Dataset(x_val, y_val) 
model = lgb.train(params = params1,
                  num_boost_round=200,
                  train_set = train_dataset, 
                  valid_sets = [train_dataset, val_dataset], 
                  verbose_eval = 50,
                  early_stopping_rounds=50,
              )

eval_df_month['pred'][val_idx] = model.predict(x_val)

#sub = df[(df.timestamp > end) & (df.timestamp < 1632181320)][['timestamp', 'Asset_ID', 'Target'] + list(df.columns[10:])].set_index('timestamp')
sub = df[(df.timestamp > end) & (df.timestamp < endv)][['timestamp', 'Asset_ID', 'Target'] + list(df.columns[10:])].set_index('timestamp')

eval_df_month['mean_pred'] = (eval_df_month['pred'] - eval_df_month.groupby('timestamp').pred.transform(np.mean)) #/ eval_df_month.Factor_Name.map(ret_df.std())
#eval_df_month['mean_pred'] = (1 * ( eval_df_month['pred'] - eval_df_month.groupby('timestamp')['pred'].transform(np.mean)) > 0)*( eval_df_month['pred'] - eval_df_month.groupby('timestamp')['pred'].transform(np.mean))
sub_eval = eval_df_month[(eval_df_month.timestamp>end) & (eval_df_month.timestamp<endv)].drop('Target', axis=1)

for col in sub.columns[2:]:
    sub[col] = sub[col] * sub.index.map(sub_eval[sub_eval.Factor_Name==col].set_index('timestamp')['mean_pred'])

print(weighted_correlation(sub.Target, sub[sub.columns[2:]].mean(axis=1), sub.Asset_ID.map(asset_df.set_index('Asset_ID')['Weight'])))

Training until validation scores don't improve for 50 rounds.
[50]	training's binary_logloss: 0.684457	valid_1's binary_logloss: 0.689644
[100]	training's binary_logloss: 0.681847	valid_1's binary_logloss: 0.689698
Early stopping, best iteration is:
[57]	training's binary_logloss: 0.684016	valid_1's binary_logloss: 0.689613
nan


In [33]:
sub = sub.dropna()
for col in sub.columns[2:]:
    print(col, weighted_correlation(sub.Target, sub[col], sub.Asset_ID.map(asset_df.set_index('Asset_ID')['Weight'])))

Return_VWAP_15 -0.0739028777178603
Return_VWAP_30 -0.06979649636213957
Return_VWAP_60 -0.06465500437106395
Return_VWAP_120 -0.06424640976269451
Return_VWAP_240 -0.05850496485523359
Vol_ratio_VWAP_100_1000 -0.026463224101277123
Vol_ratio_VWAP_1000_10000 -0.02160582392356227
Volume_ratio_100_1000 -0.03223091016001651
Volume_ratio_1000_10000 -0.035685797770807805
Count_ratio_100_1000 -0.03550258371670118
Count_ratio_1000_10000 -0.037022461142671874
BTC_ETH -0.0240760035277354
ETC_ETH -0.010250143664642134
ETHBTC_Rest_HighSize 0.0092135684551036
ETHBTC_Rest_All 0.025745331256064104


In [20]:
sub.columns[2:]

Index(['Return_VWAP_15', 'Return_VWAP_30', 'Return_VWAP_60', 'Return_VWAP_120',
       'Return_VWAP_240', 'Vol_ratio_VWAP_100_1000',
       'Vol_ratio_VWAP_1000_10000', 'Volume_ratio_100_1000',
       'Volume_ratio_1000_10000', 'Count_ratio_100_1000',
       'Count_ratio_1000_10000', 'BTC_ETH', 'ETC_ETH', 'ETHBTC_Rest_HighSize',
       'ETHBTC_Rest_All'],
      dtype='object')

In [20]:
weighted_correlation(sub.Target, sub[['Return_VWAP_15', 'Return_VWAP_30', 'Return_VWAP_60', 'Return_VWAP_120',
       ]].mean(axis=1), sub.Asset_ID.map(asset_df.set_index('Asset_ID')['Weight']))

-0.08405805396100707

In [42]:
pq = pd.read_parquet('/content/drive/MyDrive/gresearch/data/test_predictions.parquet')

In [44]:
pq = pq.reset_index()

In [46]:
pq.shape

(1792980, 6)

In [47]:
sub.index = pd.to_datetime(sub.index, unit='s')

In [49]:
sub = sub.reset_index()

In [51]:
sub[ 'pred'] = sub[['Return_VWAP_15', 'Return_VWAP_30', 'Return_VWAP_60', 'Return_VWAP_120',
       ]].mean(axis=1)

In [54]:
pq

,Asset_ID,timestamp,asset_category,Target,weights,predictions
0,3,2021-06-23 22:43:00,3,-0.000359,0.107797,-0.000153
1,2,2021-06-23 22:43:00,2,0.000824,0.058657,-0.000071
2,0,2021-06-23 22:43:00,0,0.005369,0.105286,-0.000065
3,1,2021-06-23 22:43:00,1,-0.000644,0.165850,-0.000057
4,4,2021-06-23 22:43:00,4,-0.005156,0.086971,0.000155
...,...,...,...,...,...,...
1792975,9,2021-09-20 23:44:00,9,-0.001154,0.058657,-0.000061
1792976,10,2021-09-20 23:44:00,10,0.009831,0.026874,0.000101
1792977,13,2021-09-20 23:44:00,13,0.004163,0.043830,0.000370
1792978,12,2021-09-20 23:44:00,12,0.000449,0.050867,-0.000355


In [56]:
mg = pq.merge(sub[['timestamp','Asset_ID','pred']], how='left', on=['timestamp', 'Asset_ID']).dropna()

In [70]:
weighted_correlation(mg.Target, mg['pred'], mg.Asset_ID.map(asset_df.set_index('Asset_ID')['Weight']))

-0.08794264734655757

In [ ]:
mg.pred

In [75]:
mg['ensemble'] = 0.5 * mg.predictions / (mg.pred.std() / mg.predictions.std()) - mg.pred

In [76]:
weighted_correlation(mg.Target, mg['ensemble'], mg.Asset_ID.map(asset_df.set_index('Asset_ID')['Weight']))

0.10014280360712721

In [85]:
asset_df

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar


In [86]:
for asset in mg.Asset_ID.unique():
    print(asset_df.set_index('Asset_ID').loc[asset]['Asset_Name'], mg[mg.Asset_ID==asset][['Target','ensemble']].corr()['Target']['ensemble'])

Cardano 0.08931026628271227
Bitcoin Cash 0.09511184286695862
Binance Coin 0.06461408716423038
Bitcoin 0.08371864148016314
Dogecoin 0.13717629897942982
EOS.IO 0.0605308127619332
Ethereum Classic 0.10790276995679754
Ethereum 0.061164311860459904
IOTA 0.16576973759032287
Litecoin 0.10224055675875404
Maker 0.025632550813081224
TRON 0.05842842109356866
Stellar 0.11035622334998244
Monero 0.020047646034893393
